In [1]:
import numpy as np
import tensorflow as tf
import math

In [7]:
x_data = np.random.randn(300,1)
y_data = np.random.randn(300,1)
u_x = np.random.randn(300,1)
u_y = np.random.randn(300,1)
d = np.random.randn(300,1)

dudt = np.random.randn(300,1)

In [ ]:

with tf.Session() as sess:
    # add one more lKayer and return the output of this layer
    in_size = 300
    out_size = 1
    K = tf.Variable(tf.random_normal([in_size, out_size],0,10))
    psi = tf.Variable(tf.random_normal([in_size, out_size],0,10))
    Cd = tf.Variable(tf.random_normal([in_size, out_size],0,10))
    Fdz = tf.Variable(tf.random_normal([in_size, out_size],0,10))
    Fin = tf.Variable(tf.random_normal([in_size, out_size],0,10))
    dudt_out = tf.Variable(tf.random_normal([in_size, out_size],0,10))
    outputs = tf.Variable(tf.random_normal([in_size, out_size],0,10))
    l1 = tf.Variable(tf.random_normal([in_size, out_size],0,10))
    # 添加神经网络层的函数
    def add_layer(x,y,u_x,u_y,x_y_flag,d,activation_function=None):
        if(x_y_flag == 0):
            u = u_x
        else:
            u = u_y
        # 二氧化硅运动轨迹方程的相关参数
        mu = 15
        rho_flow = 180
        rho = 2650
        g = 9.8
        Cd = 24*mu/(rho_flow*K*d)*(1/u+tf.exp(2.3288-6.5481*psi+2.4486*psi*psi))*tf.pow(rho_flow*K*u*d/mu,0.0964+0.5565*psi)/u + tf.exp(4.905-13.8944*psi+18.42222*psi*psi-10.2599*psi*psi*psi)/(1+mu/(rho_flow*K*d*u)*tf.exp(1.4681+12.2584*psi-20.7322*psi*psi+15.8855*psi*psi*psi))
        tf.print(Cd,[Cd],"Cd:")
        Fdz = math.pi*tf.pow(d,2)/8*Cd*rho_flow*tf.pow(K-1,2)*tf.pow(u,2)
        tf.print(Fdz,[Fdz],"fdz:")
        Fin = 1/6*math.pi*tf.pow(d,3)*rho*g*u/tf.pow((tf.pow(u_x,2)+tf.pow(u_y,2)),1/2)
        tf.print(Fin,[Fin],"Fin:")
        dudt_out = (Fdz - Fin)/(math.pi*tf.pow(d,3))/(rho-1/2*rho_flow*(K-1))
        if activation_function is None:
            outputs = dudt_out
        else:
            outputs = activation_function(dudt_out)
        return outputs


    # 不同时间质心序列的位置、dudt的placeholder
    x_ph = tf.placeholder(tf.float32, [None, 1])
    y_ph = tf.placeholder(tf.float32, [None, 1])
    u_x_ph = tf.placeholder(tf.float32, [None, 1])
    u_y_ph = tf.placeholder(tf.float32, [None, 1])
    d_ph = tf.placeholder(tf.float32, [None, 1])
    dudt_ph = tf.placeholder(tf.float32, [None, 1])

    l1 = add_layer(x_ph,y_ph,u_x_ph,u_y_ph,0,d_ph)
    loss = tf.reduce_mean(tf.square(outputs-dudt))
    train_step = tf.train.GradientDescentOptimizer(0.8).minimize(loss)

    #变量初始化，目的是给Graph上的图中的变量初始化。
    sess.run(tf.global_variables_initializer())
    for i in range(10000):
        sess.run(train_step,feed_dict={x_ph:x_data,y_ph:y_data,u_x_ph:u_x,u_y_ph:u_y,d_ph:d})
        if(i%100==0):
            print("loss",sess.run(loss,feed_dict={x_ph:x_data,y_ph:y_data,u_x_ph:u_x,u_y_ph:u_y,d_ph:d}))
        